### Indicadores de la Ciudad de México a partir de los resultados de la Encuesta Nacional de Ocupación y Empleo (INEGI)

Recopila todos los datos de la encuesta (21-24) y se calculan los diferentes indicadores del Sistema del Instituto de Planeación de la CDMX

In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, readxl, writexl, tools)

Cargando paquete requerido: pacman



### Descarga y lectura de datos

In [17]:
dir.create('Microdatos', showWarnings = FALSE)

#Crear carpetas para almacenar los resultados por visor del sistema
dir.create('PGD', showWarnings = FALSE)
dir.create('Visor_CDMX', showWarnings = FALSE) 


In [3]:
# Directorios principales
dir_datos <- 'Microdatos'

# Verificar si existen antes de intentar crearlos
if (!dir.exists(dir_datos)) {
    dir.create(dir_datos, recursive = TRUE)
}


# Verificar que los directorios realmente existen después de crearlos
print(file.exists(dir_datos))  # Debe devolver TRUE

[1] TRUE


In [4]:
descargar_enoe_csv <- function() {
    options(timeout = 90000)
    data_list <- list()

    descargar_extraer <- function(url, year_str, trim) {
        zip_path <- file.path('Microdatos', basename(url))
        download.file(url, zip_path, mode = 'wb')
        
        unzip_dir <- file.path('Microdatos', paste0('ENOE_', year_str, '_trim', trim))
        dir.create(unzip_dir, showWarnings = FALSE)
        unzip(zip_path, exdir = unzip_dir)

        unlink(zip_path) # Eliminar el archivo zip después de descomprimirlo
        csv_files <- list.files(unzip_dir, pattern = '\\.csv$', full.names = TRUE)
        
        if(length(csv_files) == 0) {
            warning('No se encontró el archivo CSV en ', unzip_dir)
            return(NULL) 
        }

        df_list <- list()

        for(csv_file in csv_files) {
            nombre <- file_path_sans_ext(basename(csv_file))
            tipo <- NA

            if (str_detect(nombre, 'COE1T?')) {
                tipo <- 'COE1'
                } else if (str_detect(nombre, 'COE2T?')) {
                    tipo <- 'COE2'
                    } else if (str_detect(nombre, 'HOGT?')) {
                        tipo <- 'HOG'
                        } else if (str_detect(nombre, 'SDEMT?')) {
                            tipo <- 'SDEM'
                            } else if (str_detect(nombre, 'VIVT?')) {
                                tipo <- 'VIV'
                                
                                }

            if(!is.na(tipo)) {
                df <- tryCatch({
                    read_csv(csv_file, show_col_types = FALSE) %>% clean_names()
                }, error = function(e) {
                    warning('Error al leer el archivo CSV de ', csv_file)
                    return(NULL) 
                })
                
                if(!is.null(df)) {
                    df_list[[tipo]] <- df
                }
            } 
                    
        }
        
    return(df_list)
    
    }
    
    #iterar los años y trimestres de descarga    
    for(year in 21:22) {
        year_str <- paste0('20', year)
        data_list[[year_str]] <- list()
        
        for(trim in 1:4) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20', year, '_trim', trim, '_csv.zip')
        
        df <- descargar_extraer(url, year_str, trim)

        if(!is.null(df)) {
            data_list[[year_str]][[paste0('trim', trim)]] <- df
        }
    }

    }
    
    for(year in 23:24) {
        year_str <- paste0('20', year)
        data_list[[year_str]] <- list()
        
        for(trim in 1:4) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20', year, '_trim', trim, '_csv.zip')
                
        df <- descargar_extraer(url, year_str, trim)
        
        if(!is.null(df)) {
            data_list[[year_str]][[paste0('trim', trim)]] <- df
        }
    
    }

    } 
    
    save(data_list, file = 'Microdatos/data_list.RData')


    return(data_list)

}

Advertencia: Correr la función para descargar los datos solamente una vez

In [ ]:
invisible(descargar_enoe_csv())

: 

Cargar la lista de datos guardada para no tener que descargarla nuevamente 

In [5]:
load("Microdatos/data_list.RData")

In [6]:
#Confirmar que la estructura de las listas de datos estén bien
str(data_list$'2024'$'trim4'$'SDEM')

spc_tbl_ [422,408 × 114] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ r_def     : num [1:422408] 0 0 0 0 0 0 0 0 0 0 ...
 $ loc       : logi [1:422408] NA NA NA NA NA NA ...
 $ mun       : num [1:422408] 17 13 10 15 7 5 31 NA 20 31 ...
 $ est       : num [1:422408] 30 20 30 30 20 30 20 20 40 20 ...
 $ est_d_tri : num [1:422408] 124 677 678 678 677 678 204 210 206 804 ...
 $ est_d_men : num [1:422408] 124 647 647 656 647 656 204 210 206 755 ...
 $ ageb      : num [1:422408] 0 0 0 0 0 0 0 0 0 0 ...
 $ t_loc_tri : num [1:422408] 1 1 1 1 1 1 1 3 1 1 ...
 $ t_loc_men : num [1:422408] 1 1 1 1 1 1 1 3 1 1 ...
 $ cd_a      : num [1:422408] 1 1 1 1 1 1 1 1 1 1 ...
 $ ent       : num [1:422408] 9 9 9 9 9 9 15 15 15 15 ...
 $ con       : num [1:422408] 40059 40155 40192 40204 40414 ...
 $ upm       : num [1:422408] 919939 916082 912642 917947 921598 ...
 $ d_sem     : num [1:422408] 102 406 405 410 206 210 503 503 105 206 ...
 $ n_pro_viv : num [1:422408] 65 24 80 34 100 129 48 65 389 111 ...
 $ v_

Población Económicamente Activa (PEA) de la Ciudad de México

In [56]:
PEA <- list()

In [73]:
# Función para calcular población económicamente activa
calculo_pea <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)

        pea <- df %>%
        filter(clase1 == 1) %>%
        summarise(total_pea = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_pea)

        PEA[[paste0('promedio_', year, '_', trim)]] <<- tibble(
          year = as.integer(year),
          trimestre = trim,
          total_pea = pea)

    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_pea(data_list))

In [74]:
pea_final <- bind_rows(PEA) %>%
mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
arrange(year, trimestre) 

print(pea_final)

# A tibble: 16 × 3
    year trimestre total_pea
   <int> <fct>         <dbl>
 1  2021 trim1       4175657
 2  2021 trim2       4439594
 3  2021 trim3       4724127
 4  2021 trim4       4744648
 5  2022 trim1       4658108
 6  2022 trim2       4793126
 7  2022 trim3       4798534
 8  2022 trim4       4921131
 9  2023 trim1       4876283
10  2023 trim2       4924263
11  2023 trim3       4912263
12  2023 trim4       4847179
13  2024 trim1       4910037
14  2024 trim2       5037773
15  2024 trim3       5096702
16  2024 trim4       5218209


In [76]:
write.csv(pea_final, 'Visor_CDMX/Historico_PEA.csv', row.names = FALSE)

Personal ocupado en restaurantes y servicios de alojamiento

In [25]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Turismo <- list() #Lista para vaciar los valores de población ocupada en turismo

In [26]:
# Función para calcular población ocupada y en turismo
calculo_turismo <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98) %>%
        filter(clase2 == 1, ent == 9)

      # Guardar nombres y df completo por si se necesita
      Variables[[paste0("v", year, "_", trim)]] <<- names(df)
      PobTot[[paste0("pob_total_", year, "_", trim)]] <<- df

      # Calcular turismo dentro del mismo DF
      df_turismo <- df %>%
        filter(rama_est2 == 6) %>%
        mutate(sector = "Turismo")

      total_sector <- sum(df_turismo$fac_tri, na.rm = TRUE)
      total_pob <- sum(df$fac_tri, na.rm = TRUE)
      porcentaje <- (total_sector / total_pob) * 100

      Turismo[[paste0("turismo_", year, "_", trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        sector = "Turismo",
        total_sector = total_sector,
        pob_total = total_pob,
        porcentaje = porcentaje
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_turismo(data_list))


In [27]:
# Unir resultados finales
turismo_final <- bind_rows(Turismo) %>%
  mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
  arrange(year, trimestre)

# Ver tabla final
print(turismo_final)

# A tibble: 16 × 6
    year trimestre sector  total_sector pob_total porcentaje
   <int> <fct>     <chr>          <dbl>     <dbl>      <dbl>
 1  2021 trim1     Turismo       261883   3870454       6.77
 2  2021 trim2     Turismo       293719   4117427       7.13
 3  2021 trim3     Turismo       349317   4393859       7.95
 4  2021 trim4     Turismo       391733   4465106       8.77
 5  2022 trim1     Turismo       354675   4385718       8.09
 6  2022 trim2     Turismo       402309   4534376       8.87
 7  2022 trim3     Turismo       360642   4546515       7.93
 8  2022 trim4     Turismo       381627   4698174       8.12
 9  2023 trim1     Turismo       343656   4693298       7.32
10  2023 trim2     Turismo       352408   4709095       7.48
11  2023 trim3     Turismo       379942   4712506       8.06
12  2023 trim4     Turismo       381154   4657701       8.18
13  2024 trim1     Turismo       350950   4715069       7.44
14  2024 trim2     Turismo       411774   4834239       8.52
15  2

In [10]:
write.csv(turismo_final, file = "PGD/Historico_Personal_Ocup_Restau_Serv_Aloja.csv", row.names = FALSE)

Tasa de Ocupación en la Ciudad de México

In [11]:
TasaOcupa <- list() #Lista para vaciar los valores de población ocupada en turismo


In [12]:
# Función para calcular población ocupada y en turismo
calculo_ocupacion <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)

        pob_ocupada <- df %>%
        filter(clase2 == 1) %>%
        summarise(total_ocupa = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_ocupa)

        pea <- df %>%
        filter(clase1 == 1) %>%
        summarise(total_pea = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_pea)

        tasa <- (pob_ocupada/ pea) * 100
        
        TasaOcupa[[paste0('tasa_', year, '_', trim)]] <<- tibble(
          year = as.integer(year),
          trimestre = trim,
          total_ocupa = pob_ocupada,
          total_pea = pea,
          tasa_ocupacion = tasa)

    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_ocupacion(data_list))


In [13]:
# Unir resultados finales
tasa_final <- bind_rows(TasaOcupa) %>%
  mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
  arrange(year, trimestre)

# Ver tabla final
print(tasa_final)


# A tibble: 16 × 5
    year trimestre total_ocupa total_pea tasa_ocupacion
   <int> <fct>           <dbl>     <dbl>          <dbl>
 1  2021 trim1         3870454   4175657           92.7
 2  2021 trim2         4117427   4439594           92.7
 3  2021 trim3         4393859   4724127           93.0
 4  2021 trim4         4465106   4744648           94.1
 5  2022 trim1         4385718   4658108           94.2
 6  2022 trim2         4534376   4793126           94.6
 7  2022 trim3         4546515   4798534           94.7
 8  2022 trim4         4698174   4921131           95.5
 9  2023 trim1         4693298   4876283           96.2
10  2023 trim2         4709095   4924263           95.6
11  2023 trim3         4712506   4912263           95.9
12  2023 trim4         4657701   4847179           96.1
13  2024 trim1         4715069   4910037           96.0
14  2024 trim2         4834239   5037773           96.0
15  2024 trim3         4894634   5096702           96.0
16  2024 trim4         501004

In [14]:
write.csv(tasa_final, 'Visor_CDMX/Historico_Tasa_Ocupacion.csv', row.names = FALSE)

Porcentaje del personal ocupado en servicios profesionales, científicos y técnicos

In [15]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Cientificos <- list() #Lista para vaciar los valores de población ocupada en turismo


In [16]:
calculo_cientificos <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])){
            if(!'SDEM' %in% names(data_list[[year]][[trim]])) next

        df <- data_list[[year]][[trim]] [['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1| c_res == 3), edad >= 15 & edad <= 98, clase2 == 1, ent == 9) 

        # Guardar nombres y df completo por si se necesita
        Variables[[paste0("v", year, "_", trim)]] <<- names(df)
        PobTot[[paste0("pob_total_", year, "_", trim)]] <<- df

        df_cientificos <- df %>%
        filter(scian == 12) %>%
        mutate(sector = 'Serv_prof_cientif_tecnicos')

        total_sector <- sum(df_cientificos$fac_tri, na.rm = TRUE)
        total_pob <- sum(df$fac_tri, na.rm = TRUE)
        porcentaje <- (total_sector / total_pob) * 100

        Cientificos[[paste0('Serv_prof_cientif_tecnicos_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            sector = 'Serv_prof_cientif_tecnicos',
            total_sector = total_sector,
            pob_total = total_pob,
            porcentaje = porcentaje
        )
        }
    }
}

invisible(calculo_cientificos(data_list))

In [17]:
cientificos_final <- bind_rows(Cientificos) %>%
mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
arrange(year, trimestre)

print(cientificos_final)

# A tibble: 16 × 6
    year trimestre sector                     total_sector pob_total porcentaje
   <int> <fct>     <chr>                             <dbl>     <dbl>      <dbl>
 1  2021 trim1     Serv_prof_cientif_tecnicos       238755   3870454       6.17
 2  2021 trim2     Serv_prof_cientif_tecnicos       254699   4117427       6.19
 3  2021 trim3     Serv_prof_cientif_tecnicos       266285   4393859       6.06
 4  2021 trim4     Serv_prof_cientif_tecnicos       293438   4465106       6.57
 5  2022 trim1     Serv_prof_cientif_tecnicos       286448   4385718       6.53
 6  2022 trim2     Serv_prof_cientif_tecnicos       335758   4534376       7.40
 7  2022 trim3     Serv_prof_cientif_tecnicos       345499   4546515       7.60
 8  2022 trim4     Serv_prof_cientif_tecnicos       376980   4698174       8.02
 9  2023 trim1     Serv_prof_cientif_tecnicos       356082   4693298       7.59
10  2023 trim2     Serv_prof_cientif_tecnicos       366297   4709095       7.78
11  2023 trim3     Se

In [18]:
write.csv(cientificos_final, file = 'PGD/Historico_Personal_Ocup_Serv_Prof_Cientif_Tecnicos.csv', row.names = FALSE)

Porcentaje del personal ocupado en la rama de la construcción

In [19]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Construccion <- list() #Lista para vaciar los valores de población ocupada en turismo


In [20]:
# Función para calcular población ocupada y en turismo
calculo_construccion <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98) %>%
        filter(clase2 == 1, ent == 9)

      # Guardar nombres y df completo por si se necesita
      Variables[[paste0('v', year, '_', trim)]] <<- names(df)
      PobTot[[paste0('pob_total_', year, '_', trim)]] <<- df

      # Calcular turismo dentro del mismo DF
      df_turismo <- df %>%
        filter(rama == 1) %>%
        mutate(rama = 'Construccion')

      total_rama <- sum(df_turismo$fac_tri, na.rm = TRUE)
      total_pob <- sum(df$fac_tri, na.rm = TRUE)
      porcentaje <- (total_rama / total_pob) * 100

      Construccion[[paste0('Construccion_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        rama = 'Construccion',
        total_rama = total_rama,
        pob_total = total_pob,
        porcentaje = porcentaje
      )
    }
  }
}


invisible(calculo_construccion(data_list))

In [21]:
construccion_final <- bind_rows(Construccion) %>%
mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
arrange(year, trimestre)

print(construccion_final)


# A tibble: 16 × 6
    year trimestre rama         total_rama pob_total porcentaje
   <int> <fct>     <chr>             <dbl>     <dbl>      <dbl>
 1  2021 trim1     Construccion     211301   3870454       5.46
 2  2021 trim2     Construccion     243681   4117427       5.92
 3  2021 trim3     Construccion     211700   4393859       4.82
 4  2021 trim4     Construccion     203175   4465106       4.55
 5  2022 trim1     Construccion     203653   4385718       4.64
 6  2022 trim2     Construccion     189858   4534376       4.19
 7  2022 trim3     Construccion     176073   4546515       3.87
 8  2022 trim4     Construccion     200183   4698174       4.26
 9  2023 trim1     Construccion     216016   4693298       4.60
10  2023 trim2     Construccion     206170   4709095       4.38
11  2023 trim3     Construccion     237722   4712506       5.04
12  2023 trim4     Construccion     197539   4657701       4.24
13  2024 trim1     Construccion     210541   4715069       4.47
14  2024 trim2     Co

In [36]:
write.csv(construccion_final, file = 'PGD/Historico_Personal_Ocup_Construccion.csv', row.names = FALSE)

Porcentaje del personal ocupado en servicios de esparcimiento culturales y deportivos

*Nota: Hay que cambiar la redacción y los valores del indicador en el Sistema: SI113

In [23]:
Variables <- list() 
PobTot <- list() 
Cultura <- list() 

In [24]:
calculo_cultura <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])){
            if(!'SDEM' %in% names(data_list[[year]][[trim]])) next

        df <- data_list[[year]][[trim]] [['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1| c_res == 3), edad >= 15 & edad <= 98, clase2 == 1, ent == 9) 

        # Guardar nombres y df completo por si se necesita
        Variables[[paste0('v', year, '_', trim)]] <<- names(df)
        PobTot[[paste0('pob_total_', year, '_', trim)]] <<- df

        df_cultura <- df %>%
        filter(scian == 17) %>%
        mutate(sector = 'Serv_Espar_Cultu_Depor')

        total_sector <- sum(df_cultura$fac_tri, na.rm = TRUE)
        total_pob <- sum(df$fac_tri, na.rm = TRUE)
        porcentaje <- (total_sector / total_pob) * 100

        Cultura[[paste0('Serv_Espar_Cultu_Depor_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            sector = 'Serv_Espar_Cultu_Depor',
            total_sector = total_sector,
            pob_total = total_pob,
            porcentaje = porcentaje
        )
        }
    }
}

invisible(calculo_cultura(data_list))

In [25]:
cultura_final <- bind_rows(Cultura) %>%
mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
arrange(year, trimestre)

print(cultura_final)

# A tibble: 16 × 6
    year trimestre sector                 total_sector pob_total porcentaje
   <int> <fct>     <chr>                         <dbl>     <dbl>      <dbl>
 1  2021 trim1     Serv_Espar_Cultu_Depor        23266   3870454      0.601
 2  2021 trim2     Serv_Espar_Cultu_Depor        40160   4117427      0.975
 3  2021 trim3     Serv_Espar_Cultu_Depor        50848   4393859      1.16 
 4  2021 trim4     Serv_Espar_Cultu_Depor        46448   4465106      1.04 
 5  2022 trim1     Serv_Espar_Cultu_Depor        45192   4385718      1.03 
 6  2022 trim2     Serv_Espar_Cultu_Depor        47637   4534376      1.05 
 7  2022 trim3     Serv_Espar_Cultu_Depor        66175   4546515      1.46 
 8  2022 trim4     Serv_Espar_Cultu_Depor        61754   4698174      1.31 
 9  2023 trim1     Serv_Espar_Cultu_Depor        53140   4693298      1.13 
10  2023 trim2     Serv_Espar_Cultu_Depor        67774   4709095      1.44 
11  2023 trim3     Serv_Espar_Cultu_Depor        60716   4712506     

In [37]:
write.csv(cultura_final, file = 'PGD/Historico_Personal_Ocup_Serv_Espar_Cult_Depor.csv', row.names = FALSE)

Porcentaje del personal ocupado en servicios profesionales, financieros y corporativos

In [26]:
Variables <- list() 
PobTot <- list() 
Corporativo <- list()

In [29]:
calculo_corporativo <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])){
            if(!'SDEM' %in% names(data_list[[year]][[trim]])) next

        df <- data_list[[year]][[trim]] [['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1| c_res == 3), edad >= 15 & edad <= 98, clase2 == 1, ent == 9) 

        # Guardar nombres y df completo por si se necesita
        Variables[[paste0("v", year, "_", trim)]] <<- names(df)
        PobTot[[paste0("pob_total_", year, "_", trim)]] <<- df

        df_corporativo <- df %>%
        filter(rama_est2 == 8) %>%
        mutate(sector = 'Serv_prof_finan_corpo')

        total_sector <- sum(df_corporativo$fac_tri, na.rm = TRUE)
        total_pob <- sum(df$fac_tri, na.rm = TRUE)
        porcentaje <- (total_sector / total_pob) * 100

        Corporativo[[paste0('Serv_prof_finan_corpo_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            sector = 'Serv_prof_finan_corpo',
            total_sector = total_sector,
            pob_total = total_pob,
            porcentaje = porcentaje
        )
        }
    }
}

invisible(calculo_corporativo(data_list))

In [30]:
corporativo_final <- bind_rows(Corporativo) %>%
mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
arrange(year, trimestre)


print(corporativo_final)

# A tibble: 16 × 6
    year trimestre sector                total_sector pob_total porcentaje
   <int> <fct>     <chr>                        <dbl>     <dbl>      <dbl>
 1  2021 trim1     Serv_prof_finan_corpo       662786   3870454       17.1
 2  2021 trim2     Serv_prof_finan_corpo       671689   4117427       16.3
 3  2021 trim3     Serv_prof_finan_corpo       690887   4393859       15.7
 4  2021 trim4     Serv_prof_finan_corpo       727127   4465106       16.3
 5  2022 trim1     Serv_prof_finan_corpo       688756   4385718       15.7
 6  2022 trim2     Serv_prof_finan_corpo       776295   4534376       17.1
 7  2022 trim3     Serv_prof_finan_corpo       762537   4546515       16.8
 8  2022 trim4     Serv_prof_finan_corpo       841054   4698174       17.9
 9  2023 trim1     Serv_prof_finan_corpo       872471   4693298       18.6
10  2023 trim2     Serv_prof_finan_corpo       827109   4709095       17.6
11  2023 trim3     Serv_prof_finan_corpo       827543   4712506       17.6
12  20

In [31]:
write.csv(corporativo_final, 'PGD/Historico_Personal_Ocup_Serv_Prof_Finan_Corpo.csv', row.names = FALSE)

Porcentaje del personal ocupado en la rama de la industria manufacturera

*Nota: Hay que cambiar la redacción del nombre del indicador en el Sistema: SI116

In [32]:
Variables <- list() 
PobTot <- list() 
Manufactura <- list() 

In [33]:
calculo_manufactura <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])){
            if(!'SDEM' %in% names(data_list[[year]][[trim]])) next

        df <- data_list[[year]][[trim]] [['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1| c_res == 3), edad >= 15 & edad <= 98, clase2 == 1, ent == 9) 

        # Guardar nombres y df completo por si se necesita
        Variables[[paste0("v", year, "_", trim)]] <<- names(df)
        PobTot[[paste0("pob_total_", year, "_", trim)]] <<- df

        df_manufactura <- df %>%
        filter(rama == 2) %>%
        mutate(rama = 'Manufacturera')

        total_rama <- sum(df_manufactura$fac_tri, na.rm = TRUE)
        total_pob <- sum(df$fac_tri, na.rm = TRUE)
        porcentaje <- (total_rama / total_pob) * 100

        Manufactura[[paste0('Manufacturera_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            rama = 'Manufacturera',
            total_rama = total_rama,
            pob_total = total_pob,
            porcentaje = porcentaje
        )
        }
    }
}

invisible(calculo_manufactura(data_list))

In [34]:
manufactura_final <- bind_rows(Manufactura) %>%
mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
arrange(year, trimestre)

print(manufactura_final)

# A tibble: 16 × 6
    year trimestre rama          total_rama pob_total porcentaje
   <int> <fct>     <chr>              <dbl>     <dbl>      <dbl>
 1  2021 trim1     Manufacturera     392449   3870454      10.1 
 2  2021 trim2     Manufacturera     408306   4117427       9.92
 3  2021 trim3     Manufacturera     466702   4393859      10.6 
 4  2021 trim4     Manufacturera     444658   4465106       9.96
 5  2022 trim1     Manufacturera     465524   4385718      10.6 
 6  2022 trim2     Manufacturera     473104   4534376      10.4 
 7  2022 trim3     Manufacturera     478666   4546515      10.5 
 8  2022 trim4     Manufacturera     491300   4698174      10.5 
 9  2023 trim1     Manufacturera     461489   4693298       9.83
10  2023 trim2     Manufacturera     444148   4709095       9.43
11  2023 trim3     Manufacturera     411916   4712506       8.74
12  2023 trim4     Manufacturera     454934   4657701       9.77
13  2024 trim1     Manufacturera     409430   4715069       8.68
14  20

In [35]:
write.csv(manufactura_final, 'PGD/Historico_Personal_Ocup_Indust_Manufac.csv', row.names = FALSE)

Índice de productividad económica

Promedio ponderado de los porcentajes de personal ocupado en las siguientes actividades:
manufactura; 
construcción; 
servicios de esparcimiento culturales y deportivos;
actividades en restaurantes y servicios de alojamiento; 
servicios profesionales, financieros y corporativos; 
servicios profesionales, científicos y técnicos


In [38]:
#Llamar a todos los objetos creados con datos de la ENSU en el entorno de R 
ls()

[1] "calculo_cientificos"  "calculo_construccion" "calculo_corporativo" 
 [4] "calculo_cultura"      "calculo_manufactura"  "calculo_ocupacion"   
 [7] "calculo_turismo"      "Cientificos"          "cientificos_final"   
[10] "Construccion"         "construccion_final"   "Corporativo"         
[13] "corporativo_final"    "Cultura"              "cultura_final"       
[16] "data_list"            "descargar_enoe_csv"   "dir_datos"           
[19] "Manufactura"          "manufactura_final"    "PobTot"              
[22] "tasa_final"           "TasaOcupa"            "Turismo"             
[25] "turismo_final"        "Variables"

In [39]:
# Unir todos los data frames por año y trimestre
indice <- manufactura_final %>%
  select(year, trimestre, porcentaje) %>%
  left_join(construccion_final %>% select(year, trimestre, porcentaje), by = c('year', 'trimestre')) %>%
  left_join(cultura_final %>% select(year, trimestre, porcentaje), by = c('year', 'trimestre')) %>% 
  left_join(turismo_final %>% select(year, trimestre, porcentaje), by = c('year', 'trimestre')) %>%
  left_join(corporativo_final %>% select(year, trimestre, porcentaje), by = c('year', 'trimestre')) %>%
  left_join(cientificos_final %>% select(year, trimestre, porcentaje), by = c('year', 'trimestre')) %>%
  mutate(indice = rowMeans(across(starts_with('porcentaje')), na.rm = TRUE)) %>%
  select(year, trimestre, indice)

# Ver resultado
print(indice)


# A tibble: 16 × 3
    year trimestre indice
   <int> <fct>      <dbl>
 1  2021 trim1       7.71
 2  2021 trim2       7.74
 3  2021 trim3       7.72
 4  2021 trim4       7.86
 5  2022 trim1       7.77
 6  2022 trim2       8.18
 7  2022 trim3       8.03
 8  2022 trim4       8.35
 9  2023 trim1       8.18
10  2023 trim2       8.01
11  2023 trim3       8.09
12  2023 trim4       8.00
13  2024 trim1       8.16
14  2024 trim2       8.13
15  2024 trim3       8.23
16  2024 trim4       8.23


Porcentaje de mujeres económicamente activas

In [40]:
Variables <- list() 
PEA <- list() 
Mujeres <- list() 

In [41]:
calculo_mujeres <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])) {
            if(!'SDEM' %in% names(data_list[[year]][[trim]])) next

            
        df <- data_list[[year]][[trim]] [['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1| c_res == 3), edad >= 15 & edad <= 98, clase1 == 1, ent == 9)

        # Guardar nombres y df completo por si se necesita
        Variables[[paste0("v", year, "_", trim)]] <<- names(df)
        PEA[[paste0("pob_total_", year, "_", trim)]] <<- df

        df_mujeres <- df %>%
        filter(clase1 == 1, sex == 2) %>%
        mutate(Clase = 'Mujeres_Econom_Activas')

        mujeres_pea <- sum(df_mujeres$fac_tri, na.rm = TRUE)
        pea <- sum(df$fac_tri, na.rm = TRUE)
        porcentaje <- (mujeres_pea / pea) * 100

        Mujeres[[paste0('Mujeres_Econom_Activas_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            sector = 'Mujeres_Econom_Activas',
            mujeres_pea = mujeres_pea,
            pea = pea,
            porcentaje = porcentaje
        )

        }
  
        }
    }

invisible(calculo_mujeres(data_list))    

In [43]:
mujeres_final <- bind_rows(Mujeres) %>%
mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
arrange(year, trimestre)

print(mujeres_final)

# A tibble: 16 × 6
    year trimestre sector                 mujeres_pea     pea porcentaje
   <int> <fct>     <chr>                        <dbl>   <dbl>      <dbl>
 1  2021 trim1     Mujeres_Econom_Activas     1811826 4175657       43.4
 2  2021 trim2     Mujeres_Econom_Activas     1924527 4439594       43.3
 3  2021 trim3     Mujeres_Econom_Activas     2126013 4724127       45.0
 4  2021 trim4     Mujeres_Econom_Activas     2114965 4744648       44.6
 5  2022 trim1     Mujeres_Econom_Activas     2055301 4658108       44.1
 6  2022 trim2     Mujeres_Econom_Activas     2095642 4793126       43.7
 7  2022 trim3     Mujeres_Econom_Activas     2124193 4798534       44.3
 8  2022 trim4     Mujeres_Econom_Activas     2220223 4921131       45.1
 9  2023 trim1     Mujeres_Econom_Activas     2152638 4876283       44.1
10  2023 trim2     Mujeres_Econom_Activas     2246701 4924263       45.6
11  2023 trim3     Mujeres_Econom_Activas     2185798 4912263       44.5
12  2023 trim4     Mujeres_Econo

In [44]:
write.csv(mujeres_final, 'Visor_CDMX/Historico_Mujeres_PEA.csv', row.names = FALSE)

Actualización 2: La venganza de los indicadores

Escolaridad promedio de la Ciudad de México

In [52]:
Escol_prom <- list()

In [ ]:
calculo_escolaridad <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda),
                escolar = as.numeric(anios_esc)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)

        promedio_esc <- df %>%
        filter(clase1 == 1, escolar != 99) %>%
        summarise(promedio_esc = weighted.mean(escolar, fac_tri, na.rm = TRUE)) %>%   
        pull(promedio_esc)

        
        Escol_prom[[paste0('promedio_', year, '_', trim)]] <<- tibble(
          year = as.integer(year),
          trimestre = trim,
          escolaridad_promedio = promedio_esc)

    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_escolaridad(data_list))

In [54]:
escolaridad_final <- bind_rows(Escol_prom) %>%
mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
arrange(year, trimestre)

print(escolaridad_final)

# A tibble: 16 × 3
    year trimestre escolaridad_promedio
   <int> <fct>                    <dbl>
 1  2021 trim1                     12.1
 2  2021 trim2                     12.0
 3  2021 trim3                     12.0
 4  2021 trim4                     12.1
 5  2022 trim1                     12.1
 6  2022 trim2                     12.1
 7  2022 trim3                     12.2
 8  2022 trim4                     12.3
 9  2023 trim1                     12.2
10  2023 trim2                     12.3
11  2023 trim3                     12.3
12  2023 trim4                     12.4
13  2024 trim1                     12.5
14  2024 trim2                     12.5
15  2024 trim3                     12.4
16  2024 trim4                     12.4


In [55]:
write.csv(escolaridad_final, 'Visor_CDMX/Historico_Escolaridad_Prom.csv', row.names = FALSE)

Tasa de Informalidad Laboral

In [78]:
TasaInformal <- list()

In [90]:
calculo_informal <- function(data_list) {
   for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)

        pob_ocupada <- df %>%
        filter(clase2 == 1) %>%
        summarise(total_ocupa = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_ocupa)

        informal <- df %>%
        filter(emp_ppal == 1) %>%
        summarise(total_informal = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_informal)

        tasa <- (informal/pob_ocupada) * 100
        
        TasaInformal[[paste0('tasa_', year, '_', trim)]] <<- tibble(
          year = as.integer(year),
          trimestre = trim,
          total_ocupa = pob_ocupada,
          total_informal = informal,
          tasa_informalidad = tasa)

    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_informal(data_list))


In [91]:
informal_final <- bind_rows(TasaInformal) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre)

print(informal_final)

# A tibble: 16 × 5
    year trimestre total_ocupa total_informal tasa_informalidad
   <int> <fct>           <dbl>          <dbl>             <dbl>
 1  2021 trim1         3870454        1794589              46.4
 2  2021 trim2         4117427        1939674              47.1
 3  2021 trim3         4393859        2168037              49.3
 4  2021 trim4         4465106        2144407              48.0
 5  2022 trim1         4385718        2023263              46.1
 6  2022 trim2         4534376        2180962              48.1
 7  2022 trim3         4546515        2168004              47.7
 8  2022 trim4         4698174        2152425              45.8
 9  2023 trim1         4693298        2169763              46.2
10  2023 trim2         4709095        2179459              46.3
11  2023 trim3         4712506        2191013              46.5
12  2023 trim4         4657701        2131762              45.8
13  2024 trim1         4715069        2192253              46.5
14  2024 trim2       

In [92]:
write.csv(informal_final, 'Visor_CDMX/Historico_Tasa_Informalidad.csv', row.names = FALSE)

Tasa de personal educativo

*Nota: Cambiar los valores en el sistema, ya que los cálculos anteriores no se aplicó el criterio del descriptor del INEGI:

R_DEF= 00 y C_RES=1 o 3 y EDAD 15 a 98 AÑOS

In [12]:
PobEdu <- list() 

In [20]:
calculo_Pob_Edu <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])) {
        if(!'SDEM' %in% names(data_list[[year]][[trim]])) next

        df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)

        pob_total <- df %>%
        summarise(pob_total = sum(fac_tri, na.rm = TRUE)) %>%
        pull(pob_total)

        per_educa <- df %>%
        filter(clase1 == 1, clase2 == 1, scian == 15) %>%
        summarise(per_educa = sum(fac_tri, na.rm = TRUE)) %>%
        pull(per_educa)

        tasa_edu <- (per_educa/pob_total) * 100000

        PobEdu[[paste0('edu_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            pob_total = pob_total,
            per_educa = per_educa,
            tasa_edu = tasa_edu)
        }
    } 
}

invisible(calculo_Pob_Edu(data_list))

In [21]:
pob_edu_final <- bind_rows(PobEdu) %>% 
mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
arrange(year, trimestre)

print(pob_edu_final)

# A tibble: 16 × 5
    year trimestre pob_total per_educa tasa_edu
   <int> <fct>         <dbl>     <dbl>    <dbl>
 1  2021 trim1       7746749    237063    3060.
 2  2021 trim2       7762196    267327    3444.
 3  2021 trim3       7791992    282569    3626.
 4  2021 trim4       7788791    304733    3912.
 5  2022 trim1       7752905    257385    3320.
 6  2022 trim2       7794652    294969    3784.
 7  2022 trim3       7861002    248105    3156.
 8  2022 trim4       7857997    299402    3810.
 9  2023 trim1       7878274    263152    3340.
10  2023 trim2       7898744    275598    3489.
11  2023 trim3       7903429    283793    3591.
12  2023 trim4       7935668    271716    3424.
13  2024 trim1       7996699    302597    3784.
14  2024 trim2       8015467    314024    3918.
15  2024 trim3       8031706    305198    3800.
16  2024 trim4       8003513    303630    3794.


In [22]:
write.csv(pob_edu_final, 'PGD/Historico_Tasa_Personal_Educativo.csv', row.names = FALSE)

Tasa de personal de salud y asistencia social

*Nota: Cambiar los valores en el sistema, ya que los cálculos anteriores no se aplicó el criterio del descriptor del INEGI:

R_DEF= 00 y C_RES=1 o 3 y EDAD 15 a 98 AÑOS

In [8]:
PobSalud <- list() 

In [14]:
calculo_Pob_Salud <- function(data_list) {
    for(year in names(data_list)) {
        for(trim in names(data_list[[year]])) {
        if(!'SDEM' %in% names(data_list[[year]][[trim]])) next


        df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)

        pob_total <- df %>%
        summarise(pob_total = sum(fac_tri, na.rm = TRUE)) %>%
        pull(pob_total)

        per_salud <- df %>%
        filter(clase1 == 1, clase2 == 1, scian == 16) %>%
        summarise(per_salud = sum(fac_tri, na.rm = TRUE)) %>%
        pull(per_salud)

        tasa_salud <- (per_salud/pob_total) * 100000


        PobSalud[[paste0('edu_', year, '_', trim)]] <<- tibble(
            year = as.integer(year),
            trimestre = trim,
            pob_total = pob_total,
            per_salud = per_salud,
            tasa_salud = tasa_salud)
        }
    } 
}

invisible(calculo_Pob_Salud(data_list))

In [15]:
pob_salud_final <- bind_rows(PobSalud) %>% 
mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
arrange(year, trimestre)

print(pob_salud_final)

# A tibble: 16 × 5
    year trimestre pob_total per_salud tasa_salud
   <int> <fct>         <dbl>     <dbl>      <dbl>
 1  2021 trim1       7746749    173837      2244.
 2  2021 trim2       7762196    184864      2382.
 3  2021 trim3       7791992    214159      2748.
 4  2021 trim4       7788791    214169      2750.
 5  2022 trim1       7752905    234828      3029.
 6  2022 trim2       7794652    215758      2768.
 7  2022 trim3       7861002    225968      2875.
 8  2022 trim4       7857997    228777      2911.
 9  2023 trim1       7878274    226810      2879.
10  2023 trim2       7898744    223801      2833.
11  2023 trim3       7903429    239623      3032.
12  2023 trim4       7935668    250421      3156.
13  2024 trim1       7996699    226065      2827.
14  2024 trim2       8015467    272543      3400.
15  2024 trim3       8031706    262162      3264.
16  2024 trim4       8003513    273832      3421.


In [18]:
write.csv(pob_salud_final, 'PGD/Historico_Tasa_Personal_Salud_Asisten_Social.csv', row.names = FALSE)